In [ ]:
import os
import random

import numpy as np
import cv2
import matplotlib.pyplot as plt

from skimage.color import rgb2gray
from skimage.exposure import rescale_intensity
from skimage.feature import blob_dog, blob_doh
from skimage.io import imread
from skimage.util import invert

In [ ]:
def preprocess_img(img_rgb):
    """
    This function accepts an rgb image and 
    returns a tuple containing the 
    unaltered rgb image and the preprocessed image
    """
    # make image gray
    img_gray = rgb2gray(img_rgb)
    
    # Contrast stretching
    p_start, p_end = np.percentile(img_gray, (1, 15))
    img_gray_rescale = rescale_intensity(img_gray, in_range=(p_start, p_end))
    
    # invert image
    img_gray_rescale_inverted = invert(img_gray_rescale)
    
    return (img_rgb, img_gray_rescale_inverted)


def get_preprocessed_img_from_path(img_path):
    """
    This function accepts a path to an image
    file and returns a tuple containing the 
    unaltered rgb image and the preprocessed image
    """
    # read image 
    img_rgb = imread(img_path)
    
    # preprocess image
    return preprocess_img(img_rgb)
    
    
def doh_detect_pipes(img_path=None, img=None , show=False):
    """
    This function accepts a path to an image
    file or an img file and returns the int number of pipes
    detected in the image.
    
    If the optional show param is set to true, the resuls
    are plotted visually before returning pipe counts.
    """
    # get images
    if img is None:
        img_rgb, img_preprocessed = get_preprocessed_img_from_path(img_path)
    else:
        img_rgb, img_preprocessed = preprocess_img(img)
    
    # detect pipes
    pipes = blob_doh(img_preprocessed, max_sigma=150, 
                         min_sigma=45, threshold=0.005, overlap=0.01)
    
    # plot detection
    if show:
        fig = plt.figure(dpi=300)
        ax = fig.add_subplot(1, 1, 1)
        ax.imshow(img_rgb)
        ax.set_axis_off()
        
        for pipe in pipes:
            y, x, r = pipe
            c = plt.Circle((x, y), r, color='cyan', linewidth=2, fill=False)
            ax.add_patch(c)
            
        plt.show()
    
    return len(pipes)

In [ ]:
# get all image files
imgs_dir = '../images/'
img_and_video_files = os.listdir(imgs_dir)
random.shuffle(img_and_video_files) # this makes repeat demos more interesting
img_files = filter(lambda filename: filename.endswith('.jpg') or filename.endswith('.JPG'), img_and_video_files)

# find pipes
for img_file in img_files:
    path = imgs_dir + img_file
    num_pipes = doh_detect_pipes(img_path=path, show=True)
    print(f"Found {num_pipes} in {img_file}\n\n")

In [ ]:
def detect_pipes_from_webcam():
    """
    This method takes a frame from the webcam and
    looks for pipes.
    """
    # start capture
    cap = cv2.VideoCapture(0)

    # get frame
    for _ in range(2): # this delays long enough for the mac camera to warm up
        ret, frame = cap.read()
    cap.release() # end capture

    # convert image from BGR to RGB
    frame = frame[:, :, ::-1]

    # look for pipes
    num_pipes = doh_detect_pipes(img=frame, show=True)
    print(f"Found {num_pipes} from webcam\n\n")



In [ ]:
while True:
    detect_pipes_from_webcam()